In [1]:
PATH = "/home/mdobrzanski/playground/IET/NLP/ustawy/"

In [2]:
%config Completer.use_jedi = False

In [3]:
import regex as re
import os
from morfeusz2 import Morfeusz 
import matplotlib.pyplot as plt
import elasticsearch
import elasticsearch_dsl
import os
import re

In [4]:
morf = Morfeusz()

In [5]:
es = elasticsearch.Elasticsearch()
elasticsearch_dsl.connections.add_connection('python_client', es)
es.info()

{'name': 'mdobrzanski-ASUS-TUF-Gaming-A15-FA506QM-FA506QM',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'qmsaKyaSTGuNyBjvpOr_1A',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
laws = {}

for filename in os.listdir(PATH):
    with open(PATH+filename, 'r') as file:
        tmp = file.read()
    laws[filename] = tmp

# 3

In [7]:
analyzer = elasticsearch_dsl.analyzer(
    'act_analyzer',
    type='custom',
    tokenizer='standard',                            
    filter=[
        'lowercase',                                 
        elasticsearch_dsl.analysis.token_filter(      
            'synonym_filter', 
            type='synonym', 
            synonyms=[
                'kpk,kodeks postępowania karnego',
                'kpc,kodeks postępowania cywilnego',
                'kk,kodeks karny',
                'kc,kodeks cywilny',
            ],
        ), 
        'morfologik_stem',                            
    ],
)

# 4

In [8]:
class Law(elasticsearch_dsl.Document):
    name = elasticsearch_dsl.Text()
    content = elasticsearch_dsl.Text(
            analyzer=analyzer,
        )
    
    class Index:
        name = 'laws'

In [9]:
if Law._index.exists(using=es):
    Law._index.delete(using=es)

Law.init(using=es)

# 5

In [10]:
laws.keys()

dict_keys(['1996_146.txt', '1995_427.txt', '1994_288.txt', '2004_145.txt', '2003_661.txt', '2000_28.txt', '2004_2406.txt', '2004_2699.txt', '2000_1310.txt', '2001_1068.txt', '1998_1114.txt', '1996_102.txt', '2001_41.txt', '1999_1107.txt', '2004_2580.txt', '2004_2742.txt', '1999_875.txt', '1996_602.txt', '2004_254.txt', '1999_998.txt', '1997_684.txt', '2004_2352.txt', '2004_219.txt', '1997_934.txt', '1997_297.txt', '1996_34.txt', '2004_2786.txt', '1996_493.txt', '2003_1513.txt', '2003_715.txt', '2003_507.txt', '1997_723.txt', '2003_535.txt', '1997_470.txt', '2004_1598.txt', '1996_492.txt', '2000_354.txt', '1999_728.txt', '2001_1080.txt', '1995_368.txt', '2003_1302.txt', '1996_262.txt', '1997_254.txt', '1994_602.txt', '1998_576.txt', '1998_602.txt', '2004_1206.txt', '2001_1452.txt', '1997_678.txt', '2000_1190.txt', '2001_908.txt', '2001_1116.txt', '1997_348.txt', '2000_1104.txt', '1994_294.txt', '1997_405.txt', '2004_96.txt', '2001_1645.txt', '2004_2776.txt', '2002_1802.txt', '2001_23.tx

In [11]:
for k, v in laws.items():
    law = Law(name=k, content=v)
    law.save(using=es)

# 6

In [12]:
search = elasticsearch_dsl.Search(index='laws').using(es).query('match', content='ustawa')

results = search.execute()

In [13]:
results.to_dict()

{'took': 116,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1168, 'relation': 'eq'},
  'max_score': 0.0027925,
  'hits': [{'_index': 'laws',
    '_type': '_doc',
    '_id': 'DxGe1nwBVvELDPQ8VYd0',
    '_score': 0.0027925,
    '_source': {'name': '2002_1089.txt',
     'content': '\n\n\nTekst ustawy przyjęty przez Senat bez poprawek\n\xa0\nustawa\nz dnia 7 lipca 2016 r.\no\nzmianie ustawy o działach administracji rządowej \noraz niektórych innych ustaw[1])\nArt.\n1.\nW ustawie z dnia 4 września 1997 r. o działach administracji rządowej\n(Dz.\xa0U.\xa0z\xa02016\xa0r. poz. 543 i 749) wprowadza się następujące\nzmiany:\n1)\xa0\xa0\xa0\xa0 w art. 9a w ust. 1 pkt 6\notrzymuje brzmienie:\n„6)\xa0\xa0\xa0 rewitalizacji, z\nzastrzeżeniem art. 23a pkt 2;”;\n2)\xa0\xa0\xa0\xa0 w art. 23a:\na)\xa0\xa0\xa0\xa0 pkt 2 otrzymuje brzmienie:\n„2)\xa0\xa0\xa0 programowania i\nrealizacji polityki regionalnej, obejmującej również poli

In [18]:
results = results.to_dict()

print(results['hits']['total'])

{'value': 1168, 'relation': 'eq'}


# 7

In [56]:
search = elasticsearch_dsl.Search(index='laws').using(es).query('match', content='ustawa')

results = search.count()

In [57]:
results

1178

# 8 ?

In [147]:
search = elasticsearch_dsl.Search(index='laws').using(es).query('match', content='ustawa')

results = search.count()

In [148]:
results

1178

# 9

In [23]:
search = elasticsearch_dsl.Search(index='laws').using(es).query('match_phrase',
                                                                content='kodeks postępowania cywilnego')

results = search.execute()

In [24]:
results = results.to_dict()

print(results['hits']['total'])

{'value': 99, 'relation': 'eq'}


# 10

In [41]:
search = elasticsearch_dsl.Search(index='laws').using(es)\
    .query(elasticsearch_dsl.query.MatchPhrase(content={'query': 'wchodzi w życie', 'slop': 2}))

results = search.execute()

In [42]:
results = results.to_dict()

print(results['hits']['total'])

{'value': 1174, 'relation': 'eq'}


# 11

In [88]:
search = elasticsearch_dsl.Search(index='laws').using(es)\
    .query(elasticsearch_dsl.query.Match(content={'query': 'konstytucja'}))

search = search.highlight_options(order='score')
search = search.highlight('title')

results = search.execute()

In [90]:
for hit in results[:10]:
    print(hit.name)

1997_629.txt
2000_443.txt
1997_604.txt
1996_350.txt
1997_642.txt
2001_23.txt
1996_199.txt
1999_688.txt
2001_1082.txt
1997_681.txt


# 12

In [91]:
search = elasticsearch_dsl.Search(index='laws').using(es)\
    .query(elasticsearch_dsl.query.Match(content={'query': 'konstytucja'}))

search = search.highlight_options(order='score')
search = search.highlight('content', number_of_fragments=3)

results = search.execute()

In [101]:
for hit in results:
    print(hit.name)
    for fragment in hit.meta.highlight['content']:
        print(fragment)
        print('\n')
    print('\n')

1997_629.txt
W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>


o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej


Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie




2000_443.txt
umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>


co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>


międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>




1997_604.txt
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>


Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działalności


Ciężar udowodnien